In [1]:
from sentence_transformers import SentenceTransformer
from docuverse.utils import open_stream, read_config_file
import pickle
from tqdm.autonotebook import tqdm, trange
from docuverse.utils.text_tiler import TextTiler
from docuverse.engines.search_data import SearchData
from docuverse.engines.data_template import DataTemplate
import os

In [2]:
# model_name = "thenlper/gte-small"
# model_name = "/home/raduf/mnlp-models-rag/06302024/slate.30m.english.rtrvr"
model_name = "ibm-granite/granite-embedding-30m-english"
model = SentenceTransformer(model_name)
max_size = 400
stride = 100
tiler = TextTiler(max_doc_length=max_size, stride=stride, tokenizer=model.tokenizer, aligned_on_sentences=False, trim_text_to=400)

In [8]:

# datadir="data/clapnq_small"
datadir="notebooks/data/long_example"

In [9]:
file = "Long programmatic PDF.md"
if file == "passages.tsv":
     import pandas as pd
     data = pd.read_csv(os.path.join(datadir, "passages.tsv"), sep="\t")
     sents = data['text']
elif file == "Long programmatic PDF.md":
    sents = ["".join(open_stream(os.path.join(datadir, "Long programmatic PDF.md")).readlines())]

1

In [ ]:
tiler.tokenizer.cls_token

In [ ]:
tiled_data = [tiler.create_tiles(i, txt, '', max_doc_size=512, stride=100, title_handling="none") for i, txt in enumerate(sents)]

In [ ]:
tiled_data[0][:10]

# Run the tiler in parallel

In [ ]:
from docuverse.utils import parallel_process

def proc_text(item):
     res = tiler.create_tiles(item['id'], item['text'], item['title'])
     item['result'] = res
     return item

res = parallel_process(proc_text, [{'id': i, 'text': txt, 'title': f"Title for item {i}"} for i, txt in enumerate(sents[:100])], num_threads=10)

In [14]:
res=tiler.create_tiles(id_='text0', text=sents[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


OverflowError: can't convert negative int to unsigned